In [3]:
from googleapiclient.discovery import build
from utils.google_auth import get_google_creds
import openai
import requests

# === CONFIG ===
SCOPES = [
    'https://www.googleapis.com/auth/documents',
    'https://www.googleapis.com/auth/drive'
]

from dotenv import load_dotenv
import os
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")


# === 0. Transcript Handler ===
def get_transcript(provider="manual", **kwargs):
    """
    Fetch a meeting transcript from a chosen provider.
    Providers: 'manual', 'otter', 'zoom'
    kwargs can include API keys, meeting IDs, etc.
    """
    if provider == "manual":
        print("🗣️ Paste the meeting transcript below:\n")
        return input("Transcript: ")

    elif provider == "otter":
        otter_meeting_id = kwargs.get("meeting_id")
        otter_api_key = kwargs.get("api_key")

        if not otter_meeting_id or not otter_api_key:
            raise ValueError("Missing Otter meeting_id or api_key.")

        # Placeholder Otter API call (replace with real Otter endpoint)
        # response = requests.get(
        #     f"https://api.otter.ai/v1/meetings/{otter_meeting_id}/transcript",
        #     headers={"Authorization": f"Bearer {otter_api_key}"}
        # )
        # return response.json().get("transcript", "")
        return "Simulated Otter transcript: Replace with real API call."

    elif provider == "zoom":
        zoom_meeting_id = kwargs.get("meeting_id")
        zoom_jwt_token = kwargs.get("jwt_token")

        if not zoom_meeting_id or not zoom_jwt_token:
            raise ValueError("Missing Zoom meeting_id or jwt_token.")

        # Placeholder Zoom API call (replace with real Zoom endpoint)
        # response = requests.get(
        #     f"https://api.zoom.us/v2/meetings/{zoom_meeting_id}/recordings",
        #     headers={"Authorization": f"Bearer {zoom_jwt_token}"}
        # )
        # return response.json().get("transcript", "")
        return "Simulated Zoom transcript: Replace with real API call."

    else:
        raise ValueError(f"Unsupported provider: {provider}")


# === 1. Google Docs Utilities ===
def create_doc_with_content(title, content):
    creds = get_google_creds(SCOPES)
    service = build('docs', 'v1', credentials=creds)

    doc = service.documents().create(body={'title': title}).execute()
    doc_id = doc['documentId']

    requests_payload = [{
        'insertText': {
            'location': {'index': 1},
            'text': content
        }
    }]
    service.documents().batchUpdate(
        documentId=doc_id,
        body={'requests': requests_payload}
    ).execute()

    doc_link = f"https://docs.google.com/document/d/{doc_id}/edit"
    return doc_link


# === 2. Pre-Call Briefing ===
def generate_pre_call_brief(title, attendees, agenda, bios=None, recent_threads=None, past_notes=None):
    content = f"""
📄 **Pre-Call Brief: {title}**

👥 **Attendees**: {', '.join(attendees)}

🗓️ **Agenda**:
{agenda}

🧠 **Attendee Bios**:
{bios or 'To be filled'}

📩 **Recent Email Threads**:
{recent_threads or 'No threads found.'}

📝 **Past Notes**:
{past_notes or 'No prior meeting notes found.'}
    """
    return create_doc_with_content(f"{title} - Pre-Call Brief", content.strip())


# === 3. Summarize Transcript ===
def summarize_meeting_from_transcript(transcript_text):
    prompt = f"""
You are an assistant summarizing a meeting transcript.

Transcript:
\"\"\"{transcript_text}\"\"\"

Summarize the key points, decisions made, and extract clear action items. Format with:
- Summary
- Decisions
- Action Items
    """
    response = openai.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.5
    )
    return response.choices[0].message.content


# === 4. Follow-Up Email Draft ===
def draft_follow_up_email(contact_name, summary, action_items):
    prompt = f"""
Write a friendly, professional follow-up email to {contact_name}.

Summary of the call:
{summary}

Action items:
{action_items}

Keep it clear and human. End on a warm note.
"""
    response = openai.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7
    )
    return response.choices[0].message.content


# === 5. Extract Tasks from Summary ===
def extract_tasks_from_summary(meeting_summary):
    prompt = f"""
From the following meeting summary, extract clear, actionable tasks. Format each task as:

- [Name]: [Task description]

Summary:
\"\"\"{meeting_summary}\"\"\"
"""
    response = openai.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3
    )
    return response.choices[0].message.content


# === 6. Fetch Past Notes from Google Docs ===
def fetch_latest_meeting_notes(keyword):
    creds = get_google_creds(SCOPES)
    drive_service = build('drive', 'v3', credentials=creds)
    docs_service = build('docs', 'v1', credentials=creds)

    response = drive_service.files().list(
        q=f"mimeType='application/vnd.google-apps.document' and name contains '{keyword}'",
        orderBy='modifiedTime desc',
        pageSize=1,
        fields='files(id, name)'
    ).execute()

    files = response.get('files', [])
    if not files:
        return "No matching past notes found."

    file = files[0]
    doc_id = file['id']
    doc = docs_service.documents().get(documentId=doc_id).execute()

    content = ""
    for element in doc.get('body', {}).get('content', []):
        text_run = element.get('paragraph', {}).get('elements', [{}])[0].get('textRun')
        if text_run:
            content += text_run.get('content', '')

    return content[:1500] + ("..." if len(content) > 1500 else "")


# === MAIN WORKFLOW ===
if __name__ == "__main__":
    # Step 1: Pre-call Brief Setup
    title = "Acme Q4 Planning"
    attendees = ["Jane Doe", "John Smith"]
    agenda = "- Align on Q4 marketing strategy\n- Set final revenue targets"
    bios = "- Jane: VP Marketing, Acme\n- John: Product Ops Lead"
    threads = "- Thread: Q4 launch schedule\n- Email: revenue discussion"
    
    # Fetch previous notes
    past_notes = fetch_latest_meeting_notes("Acme")
    
    # Create pre-call brief
    doc_link = generate_pre_call_brief(title, attendees, agenda, bios, threads, past_notes)
    print("✅ Pre-call brief created:", doc_link)
    
    # Step 2: Choose Transcript Provider
    provider_choice = input("Choose transcript provider (manual/otter/zoom): ").strip().lower()
    transcript = get_transcript(provider=provider_choice)
    
    # Step 3: Summarize Transcript
    summary = summarize_meeting_from_transcript(transcript)
    print("\n📝 Meeting Summary:\n", summary)
    
    # Step 4: Draft Follow-Up Email
    followup = draft_follow_up_email("Jane Doe", summary, "- Jane: Finalize proposal\n- John: Review feedback")
    print("\n📧 Follow-Up Email Draft:\n", followup)
    
    # Step 5: Extract Task List
    task_list = extract_tasks_from_summary(summary)
    print("\n✅ Task List:\n", task_list)


✅ Pre-call brief created: https://docs.google.com/document/d/1n1-RY0DB0neOEK5fvFaIdTgaM3bQiq91sfS5eanw5Nc/edit
🗣️ Paste the meeting transcript below:


📝 Meeting Summary:
 - Summary: 
The meeting was between Sarah Johnson, CEO of CloudFlow SaaS, and Akintobi Usman, an AI Solutions Consultant. Sarah shared her vision of integrating AI-driven features into CloudFlow's project management platform to enhance user engagement. The main goals include improving customer support with intelligent chatbots, predictive task recommendations, and personalized dashboards. The challenges include limited internal AI expertise, scaling AI solutions, and aligning features with customer needs. Akintobi emphasized the need for structured data pipelines, iterative feature testing, and potential partnerships for implementation. 

- Decisions: 
The team decided to move forward with identifying a single AI pilot feature, likely an intelligent chatbot for onboarding support. They also decided to schedule a tech